In [1]:
# %pip install beautifulsoup4
# %pip install scrapy
# %pip install requests

In [2]:
import scrapy
from urllib.parse import unquote
import json
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

class LinkedJobsSpider(scrapy.Spider):
    name = "linkedin_jobs"
    api_url_template = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={}&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&start={}'
    jobs_url = []

    def start_requests(self):
        start_value = 25  # Adjust the range and step based on your needs
        # keywords = ['software%20engineer', 'data%20scientist', 'data%20engineer', 'machine%20learning%20engineer', 'artificial%20intelligence%20engineer', 'python']
        keywords = 'software%2Bengineer'
        
        for start_value in range(2):
            url = self.api_url_template.format(keywords, start_value)
            yield scrapy.Request(url=url, callback=self.parse_job, meta={'start_value': start_value, 'keywords': keywords})
            start_value += 25


    def parse_job(self, response):
        start_value = response.meta.get('start_value')
        keywords = response.meta.get('keywords')
        search_keywords = unquote(keywords).replace('%2B', '-') # unquote('software%2Bengineer') -> 'software_engineer'
        jobs = response.css("li")

        num_jobs_returned = len(jobs)
        print(f"******* Num Jobs Returned for start={start_value} *******")
        print(num_jobs_returned)
        print('*****')

        for job in jobs:
            detail_url = job.css(".base-card__full-link::attr(href)").get(default='not-found').strip()
            url_parts = detail_url.split('/')
            jobs_index = url_parts.index('view')
            
            job_code = url_parts[jobs_index + 1].split("?refId")[0]
            job_id = job_code.split('-')[-1]

            # for job_link in job_links:
            yield scrapy.Request(url=detail_url, callback=self.parse_job_detail, 
                                 meta={'search_keywords' : search_keywords,'job_code': job_code, 'job_id': job_id})


    def parse_job_detail(self, response):
        job_item = {}

        print(response.text)
        script_content = response.xpath('//script[@type="application/ld+json"]/text()').get()
        soup = BeautifulSoup(script_content,'html.parser')
        description_html = soup.get_text()
        description_data = json.loads(description_html)

        # Extract job basic details
        job_item['job_id'] = response.meta.get('job_id')
        job_item['job_title'] = response.css("h1::text").get(default='not-found').strip()
        
        job_item['job_link'] = response.url
        
        job_item['company_name'] = response.css('.topcard__org-name-link::text').get(default='not-found').strip()
        job_item['company_link'] = response.css('.topcard__org-name-link::attr(href)').get(default='not-found').strip()

        job_item['job_location'] = response.css('.topcard__flavor--bullet::text').get(default='not-found').strip()

        job_item['date_posted'] = description_data['datePosted']

        job_item['job_description'] = description_data['description']

        job_item['search_keywords'] = response.meta.get('search_keywords')
        job_item['job_code'] = response.meta.get('job_code')

        self.save_to_csv(job_item)

        yield job_item

    def save_to_csv(self, item):
        # Define the output directory where CSV files will be saved
        output_dir = 'data/{}'.format(item['search_keywords'])

        # Ensure the output directory exists
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Define the CSV filename based on a unique identifier in the item
        filename = os.path.join(output_dir, f'{item["job_code"]}.csv')  # Assuming 'title' exists in the item
        df = pd.DataFrame([item])
        df.to_csv(filename, index=False)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/dz/_kg4kpk160d653__f0w9lxbm0000gn/T/ipykernel_95511/3549687521.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
settings = get_project_settings()
process = CrawlerProcess(settings)
process.crawl(LinkedJobsSpider)
process.start()

2024-02-12 21:41:23 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-02-12 21:41:23 [scrapy.utils.log] INFO: Versions: lxml 5.1.0.0, libxml2 2.12.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.9.18 (main, Sep 11 2023, 08:38:23) - [Clang 14.0.6 ], pyOpenSSL 23.2.0 (OpenSSL 3.0.12 24 Oct 2023), cryptography 41.0.7, Platform macOS-10.16-x86_64-i386-64bit
2024-02-12 21:41:23 [scrapy.addons] INFO: Enabled addons:
[]
2024-02-12 21:41:23 [py.warnings] WARNING: /Users/yidan/opt/anaconda3/envs/deeplearn/lib/python3.9/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation

******* Num Jobs Returned for start=1 *******
25
*****
******* Num Jobs Returned for start=0 *******
25
*****


2024-02-12 21:41:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.linkedin.com/jobs/view/software-engineer-remote-at-the-home-depot-3820541851?refId=VhZzswFTSVnQQhw563p91A%3D%3D&trackingId=u%2B3Tw7vSAIMDlWPgHtb5hg%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=software%2Bengineer&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&start=0)
2024-02-12 21:41:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.linkedin.com/jobs/view/software-engineer-early-career-at-lockheed-martin-3811240248?refId=nMsQ6OTJm9TO%2BlP7Qz6yJg%3D%3D&trackingId=ZU1Q%2FW2po9oYXGSVvoYC7g%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=software%2Bengineer&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-sub

<!DOCTYPE html>

    
    
    
    
    

    
    <html lang="en">
      <head>
        <meta name="pageKey" content="d_jobs_guest_details">
<!----><!---->        <meta name="locale" content="en_US">
        <meta id="config" data-app-version="2.0.1545" data-call-tree-id="AAYROystv+nYIJxy0myEaw==" data-multiproduct-name="jobs-guest-frontend" data-service-name="jobs-guest-frontend" data-browser-id="ea99f1dc-ccd8-4d05-8b0f-e2d0e8b8c648" data-enable-page-view-heartbeat-tracking data-page-instance="urn:li:page:d_jobs_guest_details;LLnNC10LR6mx/Kjs+RjuFA==" data-disable-jsbeacon-pagekey-suffix="false" data-member-id="0">

        <link rel="canonical" href="https://www.linkedin.com/jobs/view/software-engineer-remote-at-the-home-depot-3820541851">
<!----><!---->
<!---->
<!---->
          <meta property="al:android:url" content="https://www.linkedin.com/jobs/view/software-engineer-remote-at-the-home-depot-3820541851">
          <meta property="al:android:package" content="com.linkedin.andro